In [13]:
import numpy as np
import pandas as pd
import os
import re
from sqlalchemy import create_engine
from sqlalchemy.dialects.sqlite import DATE, FLOAT, TEXT
import argparse
from tqdm import tqdm

In [14]:
all_csv = os.listdir('./csv')

In [21]:
schema = {
    'ticker':TEXT,
    'date':DATE,
    'open':FLOAT,
    'high':FLOAT,
    'low':FLOAT,
    'close':FLOAT,
    'volume':FLOAT
}

In [22]:
def etl(filenames, floor):
    engine = create_engine('sqlite:///vn2.db', echo=False)
    regex = re.compile(f'.+{floor}.+')
    matches = [m for m in map(regex.match, filenames) if m is not None]
    for match in tqdm(matches):
        df = pd.read_csv('./csv/' + match.group(0))
        df = df.rename(mapper={
            '<Ticker>': 'ticker',
            '<DTYYYYMMDD>': 'date',
            '<Open>': 'open',
            '<High>': 'high',
            '<Low>': 'low',
            '<Close>': 'close',
            '<Volume>': 'volume'
        }, axis=1)
        for i in ['open', 'high', 'close', 'low']:
            df[i] = df[i].astype(float)

        df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
        df.to_sql(floor, con=engine, if_exists='append', dtype=schema, index=False)

In [23]:
etl(all_csv, 'HSX')
etl(all_csv, 'HNX')
etl(all_csv, 'UPCOM')

100%|██████████| 890/890 [01:50<00:00,  8.08it/s]


In [24]:
engine = create_engine('sqlite:///vn2.db', echo=False)

In [42]:
a = pd.read_sql("SELECT * FROM HSX ORDER BY high", con=engine)

In [43]:
a.describe()

,open,high,low,close,volume
count,568162.000000,568162.000000,568162.000000,568162.000000,5.681620e+05
mean,26.028733,26.387167,25.689072,26.075847,8.325089e+05
std,54.793858,55.221148,54.372626,54.834261,8.664047e+06
min,0.000000,0.000000,0.000000,0.010000,1.000000e+00
25%,7.900000,8.000000,7.800000,7.900000,3.950000e+03
50%,15.300000,15.600000,15.100000,15.400000,4.248000e+04
75%,29.000000,29.350000,28.500000,29.000000,2.874000e+05
max,1662.000000,1669.750000,1654.810000,1662.370000,6.178268e+08


In [45]:
a.loc[a['ticker'].str.len() == 3].describe()

,open,high,low,close,volume
count,551697.000000,551697.000000,551697.000000,551697.000000,5.516970e+05
mean,24.226270,24.576921,23.894936,24.271685,4.363429e+05
std,27.123391,27.460758,26.784007,27.148971,1.445251e+06
min,0.000000,0.380000,0.360000,0.370000,1.000000e+00
25%,8.250000,8.400000,8.100000,8.300000,3.770000e+03
50%,15.800000,16.000000,15.550000,15.900000,4.183000e+04
75%,29.300000,29.700000,28.900000,29.350000,2.918600e+05
max,342.000000,347.000000,342.000000,343.000000,1.284439e+08
